# Reconstructing LandMark Data

We need to use the [LandMark Map data](http://www.landmarkmap.org/map/) for our Land Rights layer on GFW.
These data are private and we can not copy them. We can build the layer based on the way landmark build theirs.

The data are held on an ArcGis MapServer, e.g. [http://gis.wri.org/server/rest/services/LandMark/comm_ind_Documented/MapServer](http://gis.wri.org/server/rest/services/LandMark/comm_ind_Documented/MapServer)

MapServer Query [documentation](https://developers.arcgis.com/rest/services-reference/query-map-service-layer-.htm).

In [1]:
import folium
import json
import requests
from pprint import pprint
from pyproj import Proj, transform

In [2]:
basemap = "Mapbox Bright"

map = folium.Map(location=[28.29, -16.6], zoom_start=3, tiles=basemap)

In [3]:

ind_documented = "http://gis.wri.org/server/rest/services/LandMark/comm_ind_Documented/MapServer/tile/{z}/{y}/{x}"
ind_not_documented = "http://gis.wri.org/server/rest/services/LandMark/comm_ind_NotDocumented/MapServer/tile/{z}/{y}/{x}"
ind_formal = "http://gis.wri.org/server/rest/services/LandMark/comm_ind_FormalLandClaim/MapServer/{z}/{y}/{x}"
ind_customary = "http://gis.wri.org/server/rest/services/LandMark/comm_ind_CustomaryTenure/MapServer/tile/{z}/{y}/{x}"


com_land_documented = "http://gis.wri.org/server/rest/services/LandMark/comm_comm_Documented/MapServer/tile/{z}/{y}/{x}"
com_undocumented = "http://gis.wri.org/server/rest/services/LandMark/comm_comm_NotDocumented/MapServer/tile/{z}/{y}/{x}"
com_tenure = "http://gis.wri.org/server/rest/services/LandMark/comm_comm_CustomaryTenure/MapServer/tile/{z}/{y}/{x}"
formal_lc = "http://gis.wri.org/server/rest/services/LandMark/comm_comm_FormalLandClaim/MapServer/tile/{z}/{y}/{x}"


map.add_tile_layer(tiles=ind_documented, name="Indigeouns Land: Documented", attr='LandMark', max_zoom=11, detect_retina=True)
map.add_tile_layer(tiles=ind_not_documented, name="Indigeouns Land: Not Documented", attr='LandMark', max_zoom=11, detect_retina=True)
map.add_tile_layer(tiles=ind_formal, name="Indigenous Land: Formal Tenure", attr='LandMark', max_zoom=11, detect_retina=True)
map.add_tile_layer(tiles=ind_customary, name="Indigenous Land: Customary Tenure", attr='LandMark', max_zoom=11, detect_retina=True)

map.add_tile_layer(tiles=com_land_documented, name="Community Land: Documented", attr='LandMark', max_zoom=11, detect_retina=True)
map.add_tile_layer(tiles=com_undocumented, name="Community Land: Undocumented", attr='LandMark', max_zoom=11, detect_retina=True)
map.add_tile_layer(tiles=com_tenure, name="Community Land: Custom Tenure", attr='LandMark', max_zoom=11, detect_retina=True)
map.add_tile_layer(tiles=formal_lc, name="Community Land: Formal Land Claim", attr='LandMark', max_zoom=11, detect_retina=True)


map.add_child(folium.LatLngPopup())

We also need to return metadata for this layer, and expose it via a pop-up. LandMark appear to query every Arc Layer on click to see if content exists.

An example of an original (unmodified) query from the Landmark site is below.

I have added a lat/long popup to the above to grab coordinates from the map for examples/testing.
-7.9359,-75.4555

In [4]:
example_query = ("http://gis.wri.org/server/rest/services/LandMark/comm_ind_Documented/MapServer/1/query?"
                "f=json&returnGeometry=true&spatialRel=esriSpatialRelIntersects&maxAllowableOffset=2445&"
                "geometry=%7B%22xmin%22%3A-12558297.303392%2C%22ymin%22%3A4411155.852665369%2C%22xmax%22%3A"
                "-12528945.484530501%2C%22ymax%22%3A4440507.671526869%2C%22spatialReference%22%3A%7B%22wkid%"
                "22%3A102100%7D%7D&geometryType=esriGeometryEnvelope&inSR=102100&outFields=OBJECTID%2CCountry"
                "%2CIdentity%2CForm_Rec%2CDoc_Status%2CStat_Date%2CStat_Note%2CISO_Code%2CName%2CCategory%2C"
                "Ethncty_1%2CEthncty_2%2CEthncty_3%2CPopulatn%2CPop_Source%2CPop_Year%2CArea_Ofcl%2CArea_GIS%2C"
                "Scale%2CMethod%2CData_Ctrb%2CData_Src%2CData_Date%2CAdd_Note%2CMore_info%2CUpl_Date%2C"
                "Shape_Length%2CShape_Area&outSR=102100")

r = requests.get(example_query)
print(f"Response code {r.status_code}")
pprint(r.json())

Response code 200
{'displayFieldName': 'Name',
 'features': [{'attributes': {'Add_Note': None,
                              'Area_GIS': 48931.569827286236,
                              'Area_Ofcl': 48932.1255,
                              'Category': 'Indian Reservation',
                              'Country': 'United States of America',
                              'Data_Ctrb': 'Publicly available',
                              'Data_Date': '2015',
                              'Data_Src': 'U.S. Census Bureau',
                              'Doc_Status': 'Documented',
                              'Ethncty_1': None,
                              'Ethncty_2': None,
                              'Ethncty_3': None,
                              'Form_Rec': 'Acknowledged by govt',
                              'ISO_Code': 'USA',
                              'Identity': 'Indigenous',
                              'Method': None,
                              'More_info': 'https://w

Let's break this request down and modify it to work with decimal format locations.

Also, since we dont know in advance which layer our data will be in, we need to query every layer/layer_id combination until we find features. This is the approach that LandMark seems to have taken.

https://developers.arcgis.com/rest/services-reference/query-map-service-layer-.htm

In [351]:
layers = [
    "comm_ind_Documented",
    "comm_ind_NotDocumented",
    "comm_ind_FormalLandClaim",
    "comm_ind_CustomaryTenure",
    "comm_comm_Documented",
    "comm_comm_NotDocumented",
    "comm_comm_CustomaryTenure",
    "comm_comm_FormalLandClaim"
         ]

layer_ids = [0, 1]

def make_envelope(lon, lat, offset=0.5):
    center_lon = lon
    center_lat = lat
    xmin = center_lon - offset
    xmax = center_lon + offset
    ymin = center_lat - offset
    ymax = center_lat + offset
    return {'xmin':xmin, 'ymin': ymin, 'xmax': xmax, 'ymax': ymax}

def find_popup_data(lon, lat):
    """
    Try all possible layer/layer_id combinations looking too see if any of the intersects
    contain data in the response. (This is currently what LandMark seems to do on their map site.)
    Return the dictionary object if it exists, ready to put into a pop-up!
    """
    found_items = []
    envelope = make_envelope(lon, lat, offset=0.01)  # <-- We will need to change the offset with z-level
                                                     # i.e. as we are more zoomed in, the envelope should be smaller 
    n = 1
    for layer in layers:
        for layer_id in layer_ids:
            
            print(f"{n} of {len(layer_ids) * len(layers)}. Layer: {layer}")
            n += 1
            test_query = (
                f"http://gis.wri.org/server/rest/services/LandMark/{layer}/MapServer/{layer_id}/query?"
                 "f=json&returnGeometry=true&"
                 "spatialRel=esriSpatialRelIntersects&"
                 "maxAllowableOffset=0.05&"
                f"geometry= {envelope}&"
                 "geometryType=esriGeometryEnvelope&"
                 "inSR={'wkid': 4326}&"
                 "outSR={'wkid': 4326}&"
                 "outFields=OBJECTID,Country,Identity,Form_Rec,Doc_Status,Stat_Date,Stat_Note,ISO_Code,"
                 "Name,Category,Ethncty_1,Ethncty_2,Ethncty_3,Populatn,Pop_Source,Pop_Year,Area_Ofcl,"
                 "Area_GIS,Scale,Method,Data_Ctrb,Data_Src,Data_Date,Add_Note,More_info,Upl_Date"
            )
            if layer_id == 1:
                test_query += ",Shape_Length,Shape_Area"
            try:
                r = None
                r = requests.get(test_query)
                if len(r.json().get('features')) > 0:
                    print(f"\tFound item. Name: {r.json().get('features')[0].get('attributes').get('Name')}")
                    found_items.append(r.json())
                
            except:
                pass
    return found_items


In [358]:
#found_items = find_popup_data(-111.6870, 31.9708)  # example location for polygon
#found_items = find_popup_data(-53.05220997612444, -29.40287805863888) # example location for point name Kaaguy Poty in BRA
found_items = find_popup_data(-71.7934563215134, -4.4187738539085535)


1 of 16. Layer: comm_ind_Documented
2 of 16. Layer: comm_ind_Documented
3 of 16. Layer: comm_ind_NotDocumented
4 of 16. Layer: comm_ind_NotDocumented
5 of 16. Layer: comm_ind_FormalLandClaim
6 of 16. Layer: comm_ind_FormalLandClaim
	Found item. Name: Yavarí - Mirim
7 of 16. Layer: comm_ind_CustomaryTenure
	Found item. Name: Carolina
8 of 16. Layer: comm_ind_CustomaryTenure
9 of 16. Layer: comm_comm_Documented
10 of 16. Layer: comm_comm_Documented
11 of 16. Layer: comm_comm_NotDocumented
12 of 16. Layer: comm_comm_NotDocumented
13 of 16. Layer: comm_comm_CustomaryTenure
14 of 16. Layer: comm_comm_CustomaryTenure
15 of 16. Layer: comm_comm_FormalLandClaim
16 of 16. Layer: comm_comm_FormalLandClaim


In [359]:
print(f'Found {len(found_items)} items:')

for n,item in enumerate(found_items):
    print(f'\t{n}')
    for atts in item.get('features'):
        print(f"\t\t{atts.get('attributes').get('Name')}")

Found 2 items:
	0
		Yavarí - Mirim
	1
		Carolina


In [360]:
found_items[0].get('features')

[{'attributes': {'OBJECTID': 108054,
   'Country': 'Peru',
   'Identity': 'Indigenous',
   'Form_Rec': 'Not acknowledged by govt',
   'Doc_Status': 'Held or used with formal land claim submitted',
   'Stat_Date': ' ',
   'Stat_Note': ' ',
   'ISO_Code': 'PER',
   'Name': 'Yavarí - Mirim',
   'Category': 'Reserva Indígena',
   'Ethncty_1': ' ',
   'Ethncty_2': ' ',
   'Ethncty_3': ' ',
   'Populatn': 0,
   'Pop_Source': ' ',
   'Pop_Year': 0,
   'Area_Ofcl': 0,
   'Area_GIS': 1373493.92546,
   'Scale': ' ',
   'Method': ' ',
   'Data_Ctrb': 'IBC(via RAISG)',
   'Data_Src': 'ORPIO - AIDESEP (2008)',
   'Data_Date': '2016',
   'Add_Note': ' ',
   'More_info': 'https://www.amazoniasocioambiental.org/mapas/',
   'Upl_Date': '2017/06/02',
   'Shape_Length': 784333.60588043,
   'Shape_Area': 13911607105.730444},
  'geometry': {'rings': [[[-71.57, -4.13],
     [-71.26, -4.43],
     [-71.51, -4.44],
     [-71.6, -4.53],
     [-71.62, -4.47],
     [-71.91, -4.52],
     [-71.95, -4.61],
     [-72

In [361]:
for item in found_items:
    print(item.get('features')[0].get('geometry'))
    print('')

{'rings': [[[-71.57, -4.13], [-71.26, -4.43], [-71.51, -4.44], [-71.6, -4.53], [-71.62, -4.47], [-71.91, -4.52], [-71.95, -4.61], [-72.37, -4.8], [-72.64, -5.06], [-72.77, -5.1], [-73.25, -4.8], [-73.1, -4.38], [-72.46, -4.0], [-71.57, -4.13]], [[-72.6, -4.99], [-72.6, -4.99], [-72.6, -4.99], [-72.6, -4.99]]]}

{'x': -71.79, 'y': -4.42}



In [362]:

for item in found_items:

    geojson_point = {"esrijson":item.get('features')[0].get('geometry')
                    }

    payload = geojson_point
    geostore_url = "https://production-api.globalforestwatch.org/v1/geostore"
    r = requests.post(geostore_url, json=payload)

    print('Geostore status: ',r.status_code)
    print(r.json())

Geostore status:  200
{'data': {'type': 'geoStore', 'id': '25c686b5619b688d6a46753e5020fa49', 'attributes': {'geojson': {'features': [{'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-71.57, -4.13], [-71.26, -4.43], [-71.51, -4.44], [-71.6, -4.53], [-71.62, -4.47], [-71.91, -4.52], [-71.95, -4.61], [-72.37, -4.8], [-72.64, -5.06], [-72.77, -5.1], [-73.25, -4.8], [-73.1, -4.38], [-72.46, -4], [-71.57, -4.13]]], [[[-72.6, -4.99], [-72.6, -4.99], [-72.6, -4.99], [-72.6, -4.99]]]]}}], 'crs': {}, 'type': 'FeatureCollection'}, 'hash': '25c686b5619b688d6a46753e5020fa49', 'provider': {}, 'areaHa': 1380450.1326220776, 'bbox': [-73.25, -5.1, -71.26, -4], 'lock': False, 'info': {'use': {}}}}}
Geostore status:  200
{'data': {'type': 'geoStore', 'id': '0c6ea90182ed4e822c8e698fdffd3270', 'attributes': {'geojson': {'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-71.79, -4.42]}}], 'crs': {}, 'type': 'FeatureCollection'}, 'hash': '0c6ea90182ed4

Above shows the working query, within a loop, searching a given lon, lat point for intersecting data. If the data that comes back is correct we stick it into a geostore object, which we can use to trigger an analysis.

NB we have reached the conclusion that the best way to make this work is to make the Land Right layer out of multiple layers.

This is the only way to get around the multiple simultaneous features selection bug.

In [93]:
# popup_data = json.get('features')[0].get('attributes')
# pprint(popup_data)

In [95]:
pprint(json.get('geometryType'))
pprint(json.get('spatialReference'))
pprint(json.get('features')[0].get('geometry'))

'esriGeometryPolygon'
{'latestWkid': 4326, 'wkid': 4326}
{'rings': [[[-53.24, -29.06],
            [-53.23, -29.07],
            [-53.25, -29.05],
            [-53.24, -29.06]]]}


In [ ]:
"type":"Feature",
            "properties":{

            },
            "geometry":{
               "type":"Polygon",
               "coordinates":[

In [138]:
g = {
     'type': 'Feature',
        'properties':{},
        'geometry':{
            'type':'Polygon',
            'coordinates': json.get('features')[0].get('geometry').get('rings')
        }
    }

g

{'type': 'Feature',
 'properties': {},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-111.22, 32.1],
    [-111.01, 32.15],
    [-110.96, 32.0],
    [-111.22, 32.1]],
   [[-111.41, 33.02], [-111.41, 33.02], [-111.41, 33.02], [-111.41, 33.02]],
   [[-112.72, 33.01], [-112.72, 33.01], [-112.72, 33.0], [-112.72, 33.01]],
   [[-112.73, 32.25],
    [-112.57, 32.59],
    [-111.78, 32.79],
    [-111.77, 32.37],
    [-111.58, 32.42],
    [-111.47, 32.22],
    [-111.22, 32.19],
    [-111.55, 32.01],
    [-111.62, 31.51],
    [-112.61, 31.82],
    [-112.73, 32.25]],
   [[-112.73, 32.99], [-112.71, 32.99], [-112.73, 32.99], [-112.73, 32.99]],
   [[-112.74, 33.01], [-112.73, 33.01], [-112.73, 33.01], [-112.74, 33.01]],
   [[-112.75, 33.01], [-112.75, 33.01], [-112.75, 33.01], [-112.75, 33.01]],
   [[-112.76, 33.01], [-112.76, 33.01], [-112.76, 33.0], [-112.76, 33.01]],
   [[-112.8, 33.03], [-112.8, 33.02], [-112.8, 33.02], [-112.8, 33.03]]]}}

In [139]:
geostore_url = "https://production-api.globalforestwatch.org/v1/geostore"
body = {'geojson': g}
r = requests.post(geostore_url, data=body)
print(r.url)
print(r.status_code)
print(r.json())

https://production-api.globalforestwatch.org/v1/geostore
400
{'errors': [{'source': {'parameter': 'geojson'}, 'code': 'invalid_body_parameter', 'title': 'Invalid body parameter', 'detail': 'The type property is required and was not found', 'status': 400}]}


In [137]:
g

{'type': 'Polygon',
 'coordinates': [[[-111.22, 32.1],
   [-111.01, 32.15],
   [-110.96, 32.0],
   [-111.22, 32.1]],
  [[-111.41, 33.02], [-111.41, 33.02], [-111.41, 33.02], [-111.41, 33.02]],
  [[-112.72, 33.01], [-112.72, 33.01], [-112.72, 33.0], [-112.72, 33.01]],
  [[-112.73, 32.25],
   [-112.57, 32.59],
   [-111.78, 32.79],
   [-111.77, 32.37],
   [-111.58, 32.42],
   [-111.47, 32.22],
   [-111.22, 32.19],
   [-111.55, 32.01],
   [-111.62, 31.51],
   [-112.61, 31.82],
   [-112.73, 32.25]],
  [[-112.73, 32.99], [-112.71, 32.99], [-112.73, 32.99], [-112.73, 32.99]],
  [[-112.74, 33.01], [-112.73, 33.01], [-112.73, 33.01], [-112.74, 33.01]],
  [[-112.75, 33.01], [-112.75, 33.01], [-112.75, 33.01], [-112.75, 33.01]],
  [[-112.76, 33.01], [-112.76, 33.01], [-112.76, 33.0], [-112.76, 33.01]],
  [[-112.8, 33.03], [-112.8, 33.02], [-112.8, 33.02], [-112.8, 33.03]]]}

### example query for a single point (to check size of offset applied)
```
http://gis.wri.org/server/rest/services/LandMark/comm_comm_CustomaryTenure/MapServer/0/query?f=json&returnGeometry=true&spatialRel=esriSpatialRelIntersects&maxAllowableOffset=4891&geometry={"xmin":-5981655.389737174,"ymin":-2737846.5287887035,"xmax":-5922951.752014175,"ymax":-2679142.8910657037,"spatialReference":{"wkid":102100}}&geometryType=esriGeometryEnvelope&inSR=102100&outFields=OBJECTID,Country,Identity,Form_Rec,Doc_Status,Stat_Date,Stat_Note,ISO_Code,Name,Category,Ethncty_1,Ethncty_2,Ethncty_3,Populatn,Pop_Source,Pop_Year,Area_Ofcl,Area_GIS,Scale,Method,Data_Ctrb,Data_Src,Data_Date,Add_Note,More_info,Upl_Date&outSR=102100
```

In [140]:
import pyproj

In [160]:
p1 = pyproj.Proj(init="EPSG:3857")
p2 = pyproj.Proj(init='EPSG:4326')

In [158]:
lon, lat = (-5981655.389737174, -2737846.5287887035)
#lon, lat = (-5922951.752014175, -2679142.8910657037)

# find x,y of Jefferson City, MO.
x1, y1 = p1(lon, lat)
# transform this point to projection 2 coordinates.
x2, y2 = transform(p1, p2, x1,y1)
print(x2,y2)

-180.0 90.0


In [165]:
p1(-180,1)

(-20037508.342789244, 111325.14286638486)

In [168]:
# envelope for a point
print(transform(p1, p2, -5981655.389737174, -2737846.5287887035))
print(transform(p1,p2, -5922951.752014175, -2679142.8910657037))

(-53.73412460936816, -23.87223032049987)
(-53.20678085936831, -23.389107218041758)


Looks like they are adding ~0.5 degrees around a point to create an envelope

In [171]:
transform(p1, p2,4891, 4891)

(0.043936600546285794, 0.043936596240191)

Also looks like there is an offset of 0.05 degrees added

Seems like this is only true at specific z-levels.

In [200]:

print(transform(p1, p2,-7119344.118733459, -1052868.6772707643))
print(transform(p1,p2, -7089992.299871959, -1023516.8584092644))

(-63.954156347646915, -9.415415703418075)
(-63.69048447264698, -9.155198923644756)


In [254]:
url =("http://gis.wri.org/server/rest/services/LandMark/comm_ind_FormalLandClaim/MapServer/0/query?"
    "f=json&returnGeometry=true&spatialRel=esriSpatialRelIntersects&maxAllowableOffset=2445&"
    "geometry={'xmin':-5344476.321952126,'ymin':-2868706.7212128867,'xmax':-5315124.503090627,"
              "'ymax':-2839354.902351387,'spatialReference':{'wkid':102100}}&"
    "geometryType=esriGeometryEnvelope&inSR=102100&outFields=OBJECTID,Country,Identity,"
    "Form_Rec,Doc_Status,Stat_Date,Stat_Note,ISO_Code,Name,Category,Ethncty_1,Ethncty_2,Ethncty_3,"
   "Populatn,Pop_Source,Pop_Year,Area_Ofcl,"
    "Area_GIS,Scale,Method,Data_Ctrb,Data_Src,Data_Date,Add_Note,More_info,Upl_Date&outSR=102100"
)

r = requests.get(url)
r.json().get('features')[0]

{'attributes': {'OBJECTID': 119,
  'Country': 'Brazil',
  'Identity': 'Indigenous',
  'Form_Rec': 'Not acknowledged by govt',
  'Doc_Status': 'Held or used with formal land claim submitted',
  'Stat_Date': ' ',
  'Stat_Note': None,
  'ISO_Code': 'BRA',
  'Name': 'Tekoa Pindoty',
  'Category': 'Terra Indígena',
  'Ethncty_1': None,
  'Ethncty_2': None,
  'Ethncty_3': None,
  'Populatn': 0,
  'Pop_Source': ' ',
  'Pop_Year': ' ',
  'Area_Ofcl': 0,
  'Area_GIS': 0,
  'Scale': ' ',
  'Method': ' ',
  'Data_Ctrb': 'Publicly available',
  'Data_Src': 'Fundação Nacional do Índio - FUNAI',
  'Data_Date': '2013',
  'Add_Note': None,
  'More_info': 'http://www.funai.gov.br/',
  'Upl_Date': '2015/04/23'},
 'geometry': {'x': -5327051, 'y': -2846602}}

In [261]:
print(transform(p1,p2, 2445, 2445))

(0.0219638086967223, 0.021963808158792477)


In [262]:
url =("http://gis.wri.org/server/rest/services/LandMark/comm_ind_FormalLandClaim/MapServer/0/query?"
    "f=json&returnGeometry=true&spatialRel=esriSpatialRelIntersects&maxAllowableOffset=0.0219638086967223&"
    "geometry={'xmin':-48.010247656244786,'ymin':-24.942687549135734,'xmax':-47.746575781244864,"
              "'ymax':-24.703376885973128,'spatialReference':{'wkid': 4326}}&"
    "geometryType=esriGeometryEnvelope&inSR={'wkid': 4326}&outFields=OBJECTID,Country,Identity,"
    "Form_Rec,Doc_Status,Stat_Date,Stat_Note,ISO_Code,Name,Category,Ethncty_1,Ethncty_2,Ethncty_3,"
   "Populatn,Pop_Source,Pop_Year,Area_Ofcl,"
    "Area_GIS,Scale,Method,Data_Ctrb,Data_Src,Data_Date,Add_Note,More_info,Upl_Date&outSR={'wkid': 4326}"
)

r = requests.get(url)
r.json().get('features')[0]

{'attributes': {'OBJECTID': 119,
  'Country': 'Brazil',
  'Identity': 'Indigenous',
  'Form_Rec': 'Not acknowledged by govt',
  'Doc_Status': 'Held or used with formal land claim submitted',
  'Stat_Date': ' ',
  'Stat_Note': None,
  'ISO_Code': 'BRA',
  'Name': 'Tekoa Pindoty',
  'Category': 'Terra Indígena',
  'Ethncty_1': None,
  'Ethncty_2': None,
  'Ethncty_3': None,
  'Populatn': 0,
  'Pop_Source': ' ',
  'Pop_Year': ' ',
  'Area_Ofcl': 0,
  'Area_GIS': 0,
  'Scale': ' ',
  'Method': ' ',
  'Data_Ctrb': 'Publicly available',
  'Data_Src': 'Fundação Nacional do Índio - FUNAI',
  'Data_Date': '2013',
  'Add_Note': None,
  'More_info': 'http://www.funai.gov.br/',
  'Upl_Date': '2015/04/23'},
 'geometry': {'x': -47.854, 'y': -24.763}}